In [1]:
using CuArrays

In [2]:
using DistributedArrays

## Generic algorithms: The power method as an example
The power method is a simple algorithm (not recommended for serious use!) to calculate the largest eigenvalue of a matrix.

Recall that $(\mathbf{v}, \lambda)$ is an eigenvector / eigenvalue pair for $\mathsf{A}$ if

$$\mathsf{A} \cdot \mathbf{v} = \lambda \mathbf{v},$$

where $\lambda \in \mathbb{C}$ is a scalar, which, in general, may be a complex number.

The algorithm is as follows:

> Take an arbitrary non-zero initial vector $\mathbf{v}_0$ and apply $\mathsf{A}$ to it many times, to obtain the sequence

> $$\mathbf{v}_i := \mathsf{A}^i \cdot \mathbf{v}_0.$$

This sequence of vectors converges to the eigenvector of $\mathsf{A}$ with the largest eigenvalue.

If the $\mathbf{v}_i$ are normalised (which, in practice, they usually must be so that they do not explode during the algorithm), then the eigenvalue is given by the limit of $\mathbf{v}_n \cdot \mathsf{A} \cdot \mathbf{v}_n$. In practice, with finite precision (e.g. standard `Float64`), this limit is attained after a finite number of steps.

In [3]:
M = [2 1; 1 1.]
v = [1, 1.]

2-element Array{Float64,1}:
 1.0
 1.0

In [4]:
function power_method(M, v)
    for i in 1:100
        v = M*v        # repeatedly creates a new vector and destroys the old v
        v /= norm(v)
    end
    
    return v, norm(M*v) / norm(v)  # or  (M*v) ./ v
end

power_method (generic function with 1 method)

We could write a version that *modifies* its argument `v`, if we so desired:

In [5]:
function power_method!(M, v)
    for i in 1:100
        v[:] = normalize(M*v)   # assign in-place to the elements of v
    end
    
    return v, norm(M*v) / norm(v)
end

power_method! (generic function with 1 method)

In [6]:
power_method(M, rand(2))

([0.850651, 0.525731], 2.618033988749895)

## What is generic about this?

This code is **generic**: we have not specified what kind of object `M` and `v` are, i.e. what **type** of object they are. We can now try using different types of arguments:

In [7]:
power_method(3, 1)

(1.0, 3.0)

Here, we have effectively treated the number $3$ as an operator that takes $x$ and returns $3x$.  The power method is thus a complicated way to solve the equation $3x = \lambda x$ and show that the operator $3$ has eigenvalue $3$ and "eigenvector" $1$.

### Special matrix types

In [8]:
D = Diagonal([1, 2, 3.])

3×3 Diagonal{Float64}:
 1.0   ⋅    ⋅ 
  ⋅   2.0   ⋅ 
  ⋅    ⋅   3.0

In [9]:
n = 100
S = SymTridiagonal(rand(n), rand(n-1))
S[1:5, 1:5]

5×5 Array{Float64,2}:
 0.772952  0.172261  0.0       0.0       0.0     
 0.172261  0.103728  0.688705  0.0       0.0     
 0.0       0.688705  0.193313  0.440462  0.0     
 0.0       0.0       0.440462  0.113913  0.544683
 0.0       0.0       0.0       0.544683  0.692239

In [10]:
vecS, valS = power_method(S, rand(n))

([2.8748e-16, 1.59458e-15, 3.69368e-15, 1.04023e-14, 2.79249e-14, 3.13246e-14, 6.84046e-14, 1.53086e-12, 1.46898e-10, 2.10869e-10  …  2.3669e-6, 6.2076e-6, 4.73332e-6, 3.0246e-6, 4.4592e-6, 8.77511e-6, 7.85602e-6, 1.00159e-5, 9.96368e-6, 2.91294e-6], 2.162220705846712)

Let's compare the result with the built-in Julia function `eigvals`, which, in this case, uses a function from the LAPACK library:

In [11]:
λ = eigvals(S)
valS - maximum(abs.(λ))

-4.78044297480551e-8

### Special data types

In [12]:
M = BigFloat[2 1; 1 1.]

2×2 Array{BigFloat,2}:
 2.000000000000000000000000000000000000000000000000000000000000000000000000000000  …  1.000000000000000000000000000000000000000000000000000000000000000000000000000000
 1.000000000000000000000000000000000000000000000000000000000000000000000000000000     1.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [13]:
v = [big(1.0), big(1.0)]

2-element Array{BigFloat,1}:
 1.000000000000000000000000000000000000000000000000000000000000000000000000000000
 1.000000000000000000000000000000000000000000000000000000000000000000000000000000

In [14]:
vecB, valB = power_method(M, v)

(BigFloat[8.506508083520399321815404970630110722404014037648168818367402423778840473639636e-01, 5.257311121191336060256690848478766072854979322433417815289355232412111464032098e-01], 2.618033988749894848204586834365638117720309179805762862135448622705260462818892)

For this particular matrix, an analytical expression for the eigenvalue may be obtained in the standard way, as a root of the characteristic polynomial $(\lambda - 2)(\lambda - 1) - 1 = 0$, giving

In [15]:
λ = (3 + sqrt(big(5))) / 2

2.618033988749894848204586834365638117720309179805762862135448622705260462818927

In [16]:
λ - valB

3.454467422037777850154540745120159828446400145774512554009481388067436721264971e-77

### Distributed and GPU

In [17]:
M = [2 1; 1 1.]
v = [1, 1.]

2-element Array{Float64,1}:
 1.0
 1.0

In [18]:
rmprocs(workers())

Task (done) @0x00007fb1ea2fcfd0

In [19]:
MM = CuArray(M)
vv = CuArray(v)

2-element CuArray{Float64,1}:
 1.0
 1.0

In [20]:
power_method(M, v)

([0.850651, 0.525731], 2.618033988749895)

In [21]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [22]:
@everywhere using DistributedArrays

In [23]:
dM = drand(10, 10)

10×10 DistributedArrays.DArray{Float64,2,Array{Float64,2}}:
 0.504374  0.982495   0.386204   0.360294  …  0.860438   0.444029   0.324142
 0.700654  0.314599   0.117906   0.464149     0.502569   0.972273   0.531093
 0.937465  0.031751   0.0447864  0.903031     0.260214   0.342965   0.115887
 0.498277  0.0728517  0.24941    0.687868     0.530149   0.92327    0.206641
 0.345595  0.526941   0.349415   0.632943     0.344171   0.0899879  0.388206
 0.703489  0.374165   0.98185    0.86164   …  0.476898   0.851652   0.473932
 0.762659  0.284202   0.338087   0.354351     0.266979   0.921681   0.465987
 0.374188  0.80588    0.657993   0.156958     0.374697   0.321962   0.897289
 0.295236  0.641101   0.209363   0.307681     0.955236   0.077319   0.41348 
 0.438149  0.729322   0.197634   0.418993     0.0602878  0.0112896  0.397498

In [24]:
dv = drand(10)

10-element DistributedArrays.DArray{Float64,1,Array{Float64,1}}:
 0.708128 
 0.667162 
 0.849175 
 0.336484 
 0.284678 
 0.897403 
 0.232561 
 0.988791 
 0.0776817
 0.0976372

In [25]:
dM.indexes

2×2 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:5, 1:5)   (1:5, 6:10) 
 (6:10, 1:5)  (6:10, 6:10)

In [26]:
power_method(dM, dv)

([0.380173, 0.372802, 0.259598, 0.350225, 0.249828, 0.362065, 0.313425, 0.286614, 0.314207, 0.232], 4.858036310733682)